In [1]:
print("is everything working")

is everything working


In [2]:
%pwd

'c:\\Users\\Sarbjeet Singh Pal\\Downloads\\AI-chatbot-Project\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Sarbjeet Singh Pal\\Downloads\\AI-chatbot-Project'

In [5]:
from langchain.document_loaders import CSVLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_csv_files(data):
    loader = DirectoryLoader(data, glob="*.csv", loader_cls=CSVLoader)
    documents = loader.load()
    return documents

In [7]:
extracted_data = load_csv_files("data")
extracted_data

[Document(metadata={'source': 'data\\Copy of Coffe_sales.csv', 'row': 0}, page_content='date: 01-03-2024\ndatetime: 01-03-2024 10:15:51\nhour_of_day: 10\ncash_type: card\ncard: ANON-0000-0000-0001\nmoney: R38.70\ncoffee_name: Latte\nTime_of_Day: Morning\nWeekday: Fri\nMonth_name: Mar\nWeekdaysort: 5\nMonthsort: 3'),
 Document(metadata={'source': 'data\\Copy of Coffe_sales.csv', 'row': 1}, page_content='date: 01-03-2024\ndatetime: 01-03-2024 12:19:23\nhour_of_day: 12\ncash_type: card\ncard: ANON-0000-0000-0002\nmoney: R38.70\ncoffee_name: Hot Chocolate\nTime_of_Day: Afternoon\nWeekday: Fri\nMonth_name: Mar\nWeekdaysort: 5\nMonthsort: 3'),
 Document(metadata={'source': 'data\\Copy of Coffe_sales.csv', 'row': 2}, page_content='date: 01-03-2024\ndatetime: 01-03-2024 12:20:18\nhour_of_day: 12\ncash_type: card\ncard: ANON-0000-0000-0002\nmoney: R38.70\ncoffee_name: Hot Chocolate\nTime_of_Day: Afternoon\nWeekday: Fri\nMonth_name: Mar\nWeekdaysort: 5\nMonthsort: 3'),
 Document(metadata={'sourc

In [8]:
def text_splitter(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    texts_chunks = text_splitter.split_documents(documents)
    return texts_chunks

In [9]:
text_chunks = text_splitter(extracted_data)
print(len(text_chunks))

3636


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embedding = download_embeddings()

C:\Users\Sarbjeet Singh Pal\AppData\Local\Temp\ipykernel_32332\1514844162.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Sarbjeet Singh Pal\.conda\envs\AI_chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
MODEL_API_KEY=os.getenv("MODEL_API_KEY")


os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["MODEL_API_KEY"] = MODEL_API_KEY


In [15]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [24]:
from pinecone import ServerlessSpec

index_name = "aichatbot"

if not pc.has_index(index_name):
    pc.create_index(name=index_name, dimension=384, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1"))

index = pc.Index(index_name)